In [1]:
# -*- coding: utf-8 -*-
"""
@author: Etienne Kras, open in SDB_env
"""

# imports 
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import gdal
import os
import ee
import geemap
import geojson
import datetime
import time

from geojson import Feature, FeatureCollection, dump
from shapely.geometry import Polygon
from dateutil.relativedelta import *
from google.cloud import storage
from eepackages.applications import bathymetry
from pathlib import Path
from eepackages.utils import download_image_collection, download_image_collection_thumb

# Project specific toggles

In [2]:
# see scheme at acces_api.pdf for a workflow visualization 

# project toggles
output_fol = r'p:\satellite-derived-bathymetry' # name of the local folder to store files locally
bucket = 'jip_calm_sdb' # name of the Google Cloud Storage bucket to store files in the cloud
credential_file = r'p:\11204209-jip-calm\WT4.1_SDB\GEE_images\jip-calm-c1886b3313b9.json' # Cloud Storage credential key
overall_project = 'RWS_SDB' # name of the overall project
project_name = 'Friese_Zeegat' # name of the project AoI
draw_AoI = 0 # toggle 1 to draw AoI, 0 to load

# composite image toggles
start_date = '2015-01-01' # start date of the composites
stop_date = '2020-10-01' # end date of the composites
compo_int = 3 # composite interval [months]
compo_len = 24 # composite length [months]
scale = 19.109  # output resolution of the image [m]

# Pre-processing using the API

In [4]:
# draw or load Area of Interest (AoI)

Map = geemap.Map(center=(52.643246, 5.060993), zoom=8) # initialize map with base in Hoorn

if draw_AoI == 1:
    print('Please draw a polygon somewhere in a water body') # identifier
if draw_AoI == 0:
    # open AoI
    print('Loading and visualizing AoI') #identifier
    AoIee = geemap.geojson_to_ee(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.geojson'))
    Map.addLayer(AoIee, {}, 'AoI')

Map # show map

Loading and visualizing AoI


Map(center=[52.643246, 5.060993], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [5]:
# (re)construct the AoI

if draw_AoI == 1:
    
    print('Constructing AoI from drawn polygon') # identifier
    
    # get AoI 
    AoIee = ee.FeatureCollection(Map.draw_features) # make featurecollection
    AoI = Polygon(AoIee.getInfo()['features'][0]['geometry']['coordinates'][0]) # create AoI shapefile

    # export AoI
    features = []
    features.append(Feature(geometry=AoI, properties={"AoI": project_name}))
    feature_collection = FeatureCollection(features)
    with open(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.geojson'), 'w') as f: # geojson
        dump(feature_collection, f)
    gdr = gpd.GeoDataFrame({'properties':{'AoI': project_name}, 'geometry': AoI}, crs='EPSG:4326') #shp
    gdr.to_file(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.shp'))
    bounds = ee.Geometry.Polgyon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
    
if draw_AoI == 0:
    print('Reconstructing AoI from loaded file')
    # get AoI
    with open(os.path.join(os.getcwd(),'RWS_AOI',project_name+'.geojson')) as f:
        AoIjson = geojson.load(f)
    try: # drawn polygon in this script
        AoI = Polygon(AoIjson['features'][0]['geometry']['coordinates'][0]) 
    except: # drawn in QGIS / ArcGIS and written to geojson there (client file)
        AoI = Polygon(AoIjson['features'][0]['geometry']['coordinates'][0][0])
    bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])

Reconstructing AoI from loaded file


In [7]:
# create (subtidal) composites within AoI

# image timeframes
sdate = datetime.datetime.strptime(start_date,'%Y-%m-%d')
edate = datetime.datetime.strptime(stop_date,'%Y-%m-%d')
window_length = int((edate.year-sdate.year)*12+(edate.month-sdate.month))
srangedates = pd.date_range(start_date, freq='%sMS'%(compo_int), periods=int((window_length-compo_len)/3)+1).strftime('%Y-%m-%d').tolist()
erangedates = pd.date_range((sdate+relativedelta(months=compo_len)).strftime('%Y-%m-%d'), freq='%sMS'%(compo_int), periods=int((window_length-compo_len)/3)+1).strftime('%Y-%m-%d').tolist()

sdb = bathymetry.Bathymetry() # initialize sdb instance (class)

# save composite ee.Images to a list (note, these are not yet processed)
# for intertidal assessment see for example: 
# https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/intertidal_bathymetry.ipynb
image_list = []
for starts, ends in zip(srangedates, erangedates):
    
    image = sdb.compute_inverse_depth(
        bounds=bounds,
        start=starts,
        stop=ends,
        scale=scale,
        missions=['S2', 'L8'],
        filter_masked=True,
        skip_neighborhood_search=False,
    ).clip(bounds) # clip to bounds 
    
    image_list.append(image)

In [ ]:
# show image (indicative)
# note, this takes a while as image is processed in the cloud first

AoIcenter = AoIee.geometry().centroid().coordinates()
Map = geemap.Map(center=(AoIcenter.get(1).getInfo(), AoIcenter.get(0).getInfo()), zoom=12) 
Map.addLayer(AoIee, {}, 'AoI')
Map.addLayer(image.select('red'), {}, 'composite')#, { "min": min, "max": max }, 'red-green-blue')
Map 

In [26]:
# store composites in assets or in cloud buckets (depending on your preference)
store_asset = 1 # toggle 1 to save to asset imageCollection in Google Earth Engine (user account)
store_gcs = 0 # toggle 1 to save to Google Cloud Storage (GCS) (user account), for service account GCS see for example: 
# https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/test_service_user.ipynb

# to assets --> visualize in Code Editor UI
if store_asset == 1:
    
    # get user info from the server
    user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
    asset_id = f'users/{user_name}/%s/%s'%(overall_project, project_name)

    # create folder
    try:
        ee.data.createAsset({'type': 'Folder'}, '/'.join(asset_id.split('/')[:-1]))
    except:
        print('Asset folder was already created, cannot overwrite')
    
    # create empty imageCollection
    try: 
        ee.data.createAsset({'type': 'ImageCollection'}, asset_id)
    except:
        print('Asset imageCollection was already created, cannot overwrite')
    
    # start ingesting images into imageCollection
    for img, start, end in zip(image_list, srangedates, erangedates):
        task = ee.batch.Export.image.toAsset(**{
            'image': img,
            'description': '%s_%s_%s'%(project_name, start, end),
            'scale': scale,
            'region': bounds,
            'assetId': asset_id + '/%s_%s_%s'%(project_name, start, end),
            'maxPixels': 1e11,
            'crs': 'EPSG:3857'
        })
        task.start()
        
    print('tasks submitted, check progress at: https://code.earthengine.google.com/tasks')

# to cloud storage 
if store_gcs == 1:
    for img, start, end in zip(image_list, srangedates, erangedates):
        task = ee.batch.Export.image.toCloudStorage(**{
            'image': img, 
            'description': '%s_%s_%s'%(project_name, start, end),
            'scale': scale,
            'region': bounds,
            'fileNamePrefix': '%s/%s/%s_%s_%s'%(overall_project, project_name, project_name, start, end),
            'fileFormat': 'GeoTIFF',
            'bucket': bucket, 
            'formatOptions': {'cloudOptimized': True}, # enables easy QGIS plotting
            'maxPixels': 1e11,
            'crs': 'EPSG:3857',
        })
        task.start()
        
    print('tasks submitted, check progress at: https://code.earthengine.google.com/tasks')

Asset folder was already created, cannot overwrite
tasks submitted, check progress at: https://code.earthengine.google.com/tasks


In [52]:
# store composite locally (from GCS)
local_store = 1 # toggle 1 to save to local drive

# to a local folder --> visualize in QGIS / ArcGIS (download via Cloud Storage platform or enable local storage toggle)
if local_store == 1:

    # create or check if local storage folder is present
    if not os.path.exists(os.path.join(output_fol, overall_project, project_name)):
        os.makedirs(os.path.join(output_fol, overall_project, project_name))
    
    # authentication
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_file
    
    # get file names
    client = storage.Client()
    ls = [blob for blob in client.list_blobs(bucket)] 
    
    # downloading composites to a local folder
    check_files = []
    for blob in ls:
        if project_name in blob.name:
            check_files.append(blob.name.split('/')[-1])
            blob.download_to_filename(os.path.join(output_fol, overall_project, project_name, blob.name.split('/')[-1]))
            print('Stored: ', blob.name.split('/')[-1]) # check progress
    
    # elaborate on possibility of storing locally
    if len(check_files) == 0:
        print('Please enable GCS storeing of images first, before toggling on local storage option')

Stored:  Friese_Zeegat_2015-01-01_2017-01-01.tif
Stored:  Friese_Zeegat_2015-04-01_2017-04-01.tif
Stored:  Friese_Zeegat_2015-07-01_2017-07-01.tif
Stored:  Friese_Zeegat_2015-10-01_2017-10-01.tif
Stored:  Friese_Zeegat_2016-01-01_2018-01-01.tif
Stored:  Friese_Zeegat_2016-04-01_2018-04-01.tif
Stored:  Friese_Zeegat_2016-07-01_2018-07-01.tif
Stored:  Friese_Zeegat_2016-10-01_2018-10-01.tif
Stored:  Friese_Zeegat_2017-01-01_2019-01-01.tif
Stored:  Friese_Zeegat_2017-04-01_2019-04-01.tif
Stored:  Friese_Zeegat_2017-07-01_2019-07-01.tif
Stored:  Friese_Zeegat_2017-10-01_2019-10-01.tif
Stored:  Friese_Zeegat_2018-01-01_2020-01-01.tif
Stored:  Friese_Zeegat_2018-04-01_2020-04-01.tif
Stored:  Friese_Zeegat_2018-07-01_2020-07-01.tif
Stored:  Friese_Zeegat_2018-10-01_2020-10-01.tif


In [ ]:
# enable single image downloading in the created composites.. (see for example: https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/Download_SDB_ImageCollection.ipynb)

In [ ]:
#import logging
#logging.basicConfig()
#
#sdb = bathymetry.Bathymetry() # initialize sdb instance (class)
#    
#image = sdb.compute_inverse_depth(
#    bounds=bounds,
#    start=srangedates[0],
#    stop=erangedates[0],
#    scale=scale,
#    missions=['S2', 'L8'],
#    filter_masked=True,
#    skip_neighborhood_search=False,
#).clip(bounds) # clip to bounds 
#
## create or check if local storage folder is present
#if not os.path.exists(os.path.join(output_fol, overall_project, project_name, 'single_images')):
#    os.makedirs(os.path.join(output_fol, overall_project, project_name, 'single_images'))
#    
## download and store remaining cloud-free images in the composite imageCollection
#imgCol = sdb._raw_images.map(lambda img: img.clip(bounds))
#download_image_collection(imgCol, out_dir=Path(os.path.join(output_fol, overall_project, project_name, 'single_images')), download_kwargs={"format": "GEO_TIFF", "scale": scale})

Traceback (most recent call last):
  File "C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\googleapiclient\discovery_cache\__init__.py", line 44, in autodetect
    from . import file_cache
  File "C:\Users\kras\Anaconda3\envs\SDB_env\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", lin